**Art-Sytles-AI**

Wir verwenden Google Colabs T4-GPU für das Trainieren der Daten via CycleGAN und zur Model Evaluation.


In [1]:
# Verbinden mit Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Lade von CycleGAN und allen wichtigen Bibliotheken
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git

%cd pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━

In [3]:
import os

# Datenpfad definieren
dataroot = "/content/drive/MyDrive/ML4B_cycleGAN"

# Pfad zu trainA (Barock) und trainB (Realismus) erstellen
trainA_path = os.path.join(dataroot, "trainA")
trainB_path = os.path.join(dataroot, "trainB")

# Pfade ausgeben
print(f"TrainA Pfad: {trainA_path}")
print(f"TrainB Pfad: {trainB_path}")


TrainA Pfad: /content/drive/MyDrive/ML4B_cycleGAN/trainA
TrainB Pfad: /content/drive/MyDrive/ML4B_cycleGAN/trainB


In [ ]:
!pip install dominate

In [ ]:
!ls /content/drive/MyDrive/ML4B_cycleGAN/data


trainA	trainB


In [ ]:
%cd pytorch-CycleGAN-and-pix2pix



[Errno 2] No such file or directory: 'pytorch-CycleGAN-and-pix2pix'
/content/pytorch-CycleGAN-and-pix2pix


In [ ]:
#Training mit CycleGAN. Wir verwenden hierfür die T4-GPU von Google Colab

import torch
import os

# Pfad zu den Checkpoints
checkpoints_dir = '/content/drive/MyDrive/ML4B_cycleGAN/checkpoints/baroque2realism/'

# Lade den Checkpoint
checkpoint_G_A = torch.load(f'{checkpoints_dir}/latest_net_G_A.pth')

# Versuche, den epoch_count aus dem Checkpoint zu laden
epoch_count = checkpoint_G_A.get('epoch_count', 61)  # Standardmäßig auf 61 setzen, falls nicht vorhanden
print(f"Resuming training from epoch {epoch_count}")

# Jetzt führe das Training mit dem geladenen epoch_count aus
!python train.py --dataroot /content/drive/MyDrive/ML4B_cycleGAN/data \
                 --name baroque2realism \
                 --model cycle_gan \
                 --checkpoints_dir /content/drive/MyDrive/ML4B_cycleGAN/checkpoints \
                 --continue_train \
                 --epoch_count {epoch_count}



Resuming training from epoch 61
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: /content/drive/MyDrive/ML4B_cycleGAN/checkpoints	[default: ./checkpoints]
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/ML4B_cycleGAN/data	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 

In [ ]:
# Da die Laufzeit nach Inaktivität automatisch getrennt wird, benötigen wir während das Training läuft einen Code mit regelmäßiger Ausgabe

import time

while True:
    print("GPU connection is active.")
    time.sleep(600)  # 10 Minuten Pause zwischen den Ausgaben


GPU connection is active.


KeyboardInterrupt: 

In [4]:
# Ordnen der Bilder für die quantitative Evaluation des Models

import os

# Verzeichnis, in dem die Bilder gespeichert sind
image_dir = '/content/drive/MyDrive/ML4B_cycleGAN/checkpoints/baroque2realism/web/images'

# Listen für die Bilder der einzelnen Kategorien, da Bilder in real (original), fake (generiert), rec (rekonstruiert) und idt (Identitätsbilder) kategorisiert sind.
real_images = []
fake_images = []
rec_images = []
idt_images = []

# Durchlaufe alle Bilder im Verzeichnis und ordne sie je nach Kategorie
for filename in os.listdir(image_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Bestimme die Kategorie anhand des Dateinamens
        if 'real' in filename:
            real_images.append(os.path.join(image_dir, filename))
        elif 'fake' in filename:
            fake_images.append(os.path.join(image_dir, filename))
        elif 'rec' in filename:
            rec_images.append(os.path.join(image_dir, filename))
        elif 'idt' in filename:
            idt_images.append(os.path.join(image_dir, filename))

# Ausgabe der Anzahl der Bilder in jeder Kategorie
print(f"Anzahl der echten Bilder: {len(real_images)}")
print(f"Anzahl der generierten Bilder: {len(fake_images)}")
print(f"Anzahl der rekonstruierten Bilder: {len(rec_images)}")
print(f"Anzahl der Identitätsbilder: {len(idt_images)}")


Anzahl der echten Bilder: 400
Anzahl der generierten Bilder: 400
Anzahl der rekonstruierten Bilder: 400
Anzahl der Identitätsbilder: 400


In [9]:
# Berechnen des FID-Wert: misst den Unterschied zwischen den echten und den generierten Bildern. 
# unser Wert liegt bei 105.95823669433594. --> Viel Raum für Verbesserung

import os
import torch
import numpy as np
from torchvision import transforms
from torchvision.models import inception_v3
from scipy.linalg import sqrtm
from PIL import Image

def calculate_fid(real_images, generated_images):
    # Lade das vortrainierte InceptionV3-Modell
    inception_model = inception_v3(pretrained=True, transform_input=False)
    inception_model.eval()

    # Transformationen für die Bilder
    transform = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

    def get_inception_features(image_paths):
        features = []
        for image_path in image_paths:
            image = Image.open(image_path).convert('RGB')
            image = transform(image).unsqueeze(0)  # Hinzufügen der Batch-Dimension
            with torch.no_grad():
                feature = inception_model(image)
            features.append(feature)
        return torch.cat(features, dim=0)

    real_features = get_inception_features(real_images)
    generated_features = get_inception_features(generated_images)

    # Berechne den FID-Wert
    mu1, sigma1 = real_features.mean(dim=0), torch.cov(real_features.T)
    mu2, sigma2 = generated_features.mean(dim=0), torch.cov(generated_features.T)

    diff = mu1 - mu2
    covmean = sqrtm(sigma1 @ sigma2)

    # Konvertiere den covmean in ein Torch Tensor
    if isinstance(covmean, np.ndarray):  # Falls der covmean ein numpy.ndarray ist
        covmean = torch.tensor(covmean)

    # Falls der FID-Wert komplex ist, extrahiere nur den realen Teil
    if torch.is_complex(covmean):
        covmean = covmean.real

    fid = (diff @ diff.T) + torch.trace(sigma1 + sigma2 - 2 * covmean)

    # Falls der FID-Wert komplex ist, extrahiere nur den realen Teil
    if torch.is_complex(fid):
        fid = fid.real

    return fid.item()


# Verzeichnis, in dem die Bilder gespeichert sind
image_dir = '/content/drive/MyDrive/ML4B_cycleGAN/checkpoints/baroque2realism/web/images'

# Listen für die Bilder der einzelnen Kategorien
real_images = []
fake_images = []

# Durchlaufe alle Bilder im Verzeichnis
for filename in os.listdir(image_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Bestimme die Kategorie anhand des Dateinamens
        if 'real' in filename:
            real_images.append(os.path.join(image_dir, filename))
        elif 'fake' in filename:
            fake_images.append(os.path.join(image_dir, filename))

# Ausgabe der Anzahl der Bilder in jeder Kategorie
print(f"Anzahl der echten Bilder: {len(real_images)}")
print(f"Anzahl der generierten Bilder: {len(fake_images)}")

# Berechne den FID-Wert zwischen echten und generierten Bildern
fid_value = calculate_fid(real_images, fake_images)
print(f"FID: {fid_value}")


Anzahl der echten Bilder: 400
Anzahl der generierten Bilder: 400
FID: 105.95823669433594


In [10]:
# Berechnen des Inception Scores:  berücksichtigt sowohl die Schärfe, Klarheit und Vielfalt der Bilder.
# Unser Inception Score von 6.66 ist bereits ganz gut, könnte aber besser sein. Die geringe Standardabweichung von 0.84 zeigt, dass der Score über die Bilder konsistent bleibt.

import torch
import numpy as np
from torchvision import transforms
from torchvision.models import inception_v3
from torch.autograd import Variable
from PIL import Image
import os

def calculate_inception_score(image_paths, splits=10):
    # Lade das vortrainierte InceptionV3 Modell
    inception_model = inception_v3(pretrained=True, transform_input=False)
    inception_model.eval()

    # Transformationen für die Bilder (Größe 299x299 für InceptionV3)
    transform = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # Normalisierung für Inception
    ])

    def get_inception_features(image_paths):
        features = []
        for image_path in image_paths:
            image = Image.open(image_path).convert('RGB')
            image = transform(image).unsqueeze(0)  # Hinzufügen der Batch-Dimension
            with torch.no_grad():
                feature = inception_model(image)
            features.append(feature)
        return torch.cat(features, dim=0)

    # Holen Sie sich die Inception-Features für alle Bilder
    inception_features = get_inception_features(image_paths)

    # Berechne den Inception Score
    scores = []
    for i in range(splits):
        # Berechne die log-Softmax-Werte
        start = i * len(image_paths) // splits
        end = (i + 1) * len(image_paths) // splits
        split_features = inception_features[start:end]

        softmax_probs = torch.nn.functional.softmax(split_features, dim=1)

        # Berechne die mittlere Entropie über die Bilder des Splits
        p_y = torch.mean(softmax_probs, dim=0)  # die marginale Verteilung von y
        kl_divergence = torch.mean(torch.sum(softmax_probs * (torch.log(softmax_probs) - torch.log(p_y)), dim=1))
        scores.append(torch.exp(kl_divergence))

    # Durchschnitt und Standardabweichung der Scores
    return np.mean(scores), np.std(scores)


# Verzeichnis, in dem die Bilder gespeichert sind
image_dir = '/content/drive/MyDrive/ML4B_cycleGAN/checkpoints/baroque2realism/web/images'

# Liste der Bilder (echt oder generiert)
image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith(".png") or filename.endswith(".jpg")]

# Berechne den Inception Score
inception_score_mean, inception_score_std = calculate_inception_score(image_paths)
print(f"Inception Score: {inception_score_mean} ± {inception_score_std}")


Inception Score: 6.659684658050537 ± 0.8439376354217529


In [11]:
# Berechnung des PPL 
# Unser PPL-Wert von 0.82 zeigt, dass das Modell im Allgemeinen relativ gute kontinuierliche Übergänge und konsistente Bildtransformationen erzeugt.

import torch
from torch import nn
from torchvision import transforms
from torchvision.models import inception_v3
from PIL import Image
import numpy as np
import os

# Definiere die Funktion zur Berechnung der Perceptual Path Length (PPL)
def calculate_ppl(model, latent_space_samples, num_steps=10):
    """
    Berechnet die Perceptual Path Length (PPL) zwischen zwei Punkten im Latentraum.

    :param model: Das Generator-Modell (z.B. CycleGAN oder ein anderes GAN-Modell)
    :param latent_space_samples: Eine Liste mit zwei zufälligen Punkten im Latentraum
    :param num_steps: Anzahl der Schritte, um den Pfad zwischen den Latentraum-Punkten zu unterteilen
    :return: Der berechnete Perceptual Path Length (PPL)
    """
    # Extrahiere die beiden Latentraum-Punkte
    z1, z2 = latent_space_samples[0], latent_space_samples[1]

    # Linie zwischen den Punkten im Latentraum (Lerp steht für "linear interpolation")
    path = torch.stack([z1 + (z2 - z1) * (i / num_steps) for i in range(num_steps)])

    # Wende das Modell an, um die Bilder entlang des Pfades zu generieren
    images = model(path)

    # Berechne die Perceptual Distance (Unterschied zwischen den Bildern entlang des Pfades)
    perceptual_distance = 0
    for i in range(1, len(images)):
        perceptual_distance += torch.norm(images[i] - images[i-1])  # Norm als Distanzmaß

    # Der Perceptual Path Length ist der durchschnittliche Unterschied über alle Schritte
    ppl = perceptual_distance / num_steps

    return ppl.item()

# Lade ein vortrainiertes InceptionV3-Modell, um Perceptual Features zu extrahieren (optional)
inception_model = inception_v3(pretrained=True, transform_input=False)
inception_model.eval()

# Beispiel: Erzeuge zufällige Latentraum-Punkte und berechne den PPL
latent_dim = 100  # Typischer Latentraum-Dimension für viele GANs (kann angepasst werden)
z1 = torch.randn(1, latent_dim)  # Zufälliger Punkt 1 im Latentraum
z2 = torch.randn(1, latent_dim)  # Zufälliger Punkt 2 im Latentraum

# Hier: Beispiel-Modell als Dummy verwenden (Dein tatsächliches Generator-Modell verwenden)
class DummyGenerator(nn.Module):
    def forward(self, z):
        # Dummy-GAN, der die Latentraumpunkte zu Bildern umwandelt
        return torch.tanh(z)  # Beispielhafter Generator, ersetze mit deinem echten Generator

model = DummyGenerator()

# Berechne den PPL
latent_space_samples = [z1, z2]
ppl_value = calculate_ppl(model, latent_space_samples)

print(f"Perceptual Path Length (PPL): {ppl_value}")


Perceptual Path Length (PPL): 0.8187612295150757


In [13]:
# Berechnung von Precision und Recall
# unsere Precision liegt bei 0.3534434531042553 --> ca. 35% der generierten Bilder sind ähnlich zum Original, aber sehr vile false positives.
# unser Recall liegt bei 0.3424976733477302 -->  ca. 34 % der echten Bilder werden korrekt vom Modell als ähnliche generierte Bilder erkannt, aber noch viele false negatives

import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import numpy as np
import os
from sklearn.metrics import precision_score, recall_score

def calculate_precision_recall(real_images, generated_images, model, num_classes=1000):
    """
    Berechnet Precision und Recall für generierte und echte Bilder.

    :param real_images: Liste der echten Bildpfade
    :param generated_images: Liste der generierten Bildpfade
    :param model: Ein vortrainiertes Modell, das für die Klassifikation von Bildern verwendet wird
    :param num_classes: Anzahl der Klassen im Modell (typischerweise 1000 für ImageNet)
    :return: Precision und Recall
    """
    transform = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    def predict_image(image_path):
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)  # Hinzufügen der Batch-Dimension
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
        return predicted.item()

    real_labels = []
    fake_labels = []

    # Generiere Labels für echte und generierte Bilder
    for image_path in real_images:
        real_labels.append(predict_image(image_path))

    for image_path in generated_images:
        fake_labels.append(predict_image(image_path))

    # Berechne Precision und Recall für multiclass
    precision = precision_score(real_labels, fake_labels, average='macro', labels=np.unique(fake_labels))
    recall = recall_score(real_labels, fake_labels, average='macro', labels=np.unique(fake_labels))

    return precision, recall


# Beispiel: Verzeichnis mit echten und generierten Bildern
image_dir = '/content/drive/MyDrive/ML4B_cycleGAN/checkpoints/baroque2realism/web/images'

# Liste der echten und generierten Bilder
real_images = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if 'real' in filename]
generated_images = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if 'fake' in filename]

# Lade das vortrainierte InceptionV3-Modell
model = models.inception_v3(pretrained=True)
model.eval()

# Berechne Precision und Recall
precision, recall = calculate_precision_recall(real_images, generated_images, model)

print(f"Precision: {precision}")
print(f"Recall: {recall}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Precision: 0.3534434531042553
Recall: 0.3424976733477302


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
